In [4]:
'''
Create the visualization gui for PropSim
Author: Joshua G. Albert albert@strw.leidenuniv.nl
'''

#from PropSim import toolkit as propSimTk

#%matplotlib inline
import numpy as np
import pylab as plt
from sys import stdout
from matplotlib.widgets import RadioButtons, Cursor, Slider, Button
from TextBoxWidget import TextBox

class visual(object):
    def __init__(self,array):
        self.array = array
        self.log("Visualizing Array: {0}".format(array))
        #self.tk = propSimTk(array=self.array)
        self.minLayer = 0
        self.maxLayer = 100
        self.curWavelength = 0.21
        self.curLayer = self.maxLayer
        self.curLayerHeight = self.getCurLayerHeight()
        self.dataSelection = ['Visibilities <E*E>',
                              'Intensity <E.E>',
                              'Optical Thickness',
                              'Electron Density',
                              'Refractive Index']
        self.dataSelectionCallers = {'Visibilities <E*E>':None,
                                     'Intensity <E.E>':None,
                                     'Optical Thickness':None,
                                     'Electron Density':None,
                                     'Refractive Index':None}
        self.fig = None
    def log(self,message):
        stdout.write("{0}\n".format(message))
        stdout.flush()
        
    def createGui(self):
        self.fig = None
        self.infoDisplayAxes = None
        #layers infoax
        self.layerSliderAxes = None
        self.layerInfoAxes = None
        self.layerBottomAxes = None
        self.layerDownAxes = None
        self.layerTopAxes = None
        self.layerUpAxes = None
        self.layerTopAxes = None
        self.wavelengthAxes = None
        self.frequencyAxes = None
        self.dataSelectionAxes = None
        self.imageAxes = None
        

        self.resetGui()
    def getAspect(self):
        '''Return image aspect for equidistant coordiantes'''
        #get xlim, ylim
        #return ylim/xlim
        return 1.
    def resizeCb(self,event):
        self.figW = event.width/self.dpi
        self.figH = event.height/self.dpi
        #print self.figW,self.figH
        self.resetGui()
    def resetGui(self):
        '''Builds the gui or rescales things if not yet built'''
        if self.fig is None:
            self.figH = 6#in
            self.figW = 6#in
            self.dpi = float(90)#pt/in
            self.fig = plt.figure(figsize=[self.figW,self.figH],dpi=self.dpi,facecolor='grey')
            plt.connect('resize_event',self.resizeCb)
        self.padding = 0.02#rel
        self.font = 12#pt
        self.lineHeight = self.font*1.2/(self.figH*self.dpi)#rel
        self.titleLineHeight = self.lineHeight#rel
        self.titleLineWidth = 1. - 2*self.padding
        self.titleLineBL = (self.padding, 1-self.padding - self.titleLineHeight)
        self.infoDisplayWidth = min(self.font*23/(self.figW*self.dpi),1./3.)#rel
        self.infoDisplayHeight = 1. - 3*self.padding - self.titleLineHeight#rel
        self.infoDisplayBL = (self.padding, self.padding)#rel
        self.imageYlabelMargin = self.font*2.5/(self.figW*self.dpi)#rel
        self.imageXlabelMargin = self.font*2.5/(self.figH*self.dpi)#rel
        self.imageAxesWidth = 1. - 3*self.padding - self.infoDisplayWidth - self.imageYlabelMargin#rel
        self.imageAxesAspect = self.getAspect()#*self.figW/self.figH#h/w
        self.imageAxesHeight = self.imageAxesWidth*self.imageAxesAspect#rel
        self.dataSelectionWidth = self.imageAxesWidth + self.imageYlabelMargin#rel
        self.dataSelectionHeight = 1. - 4*self.padding - self.titleLineHeight - self.imageAxesHeight - self.imageXlabelMargin#rel
        self.imageAxesBL = (2*self.padding + self.infoDisplayWidth + self.imageYlabelMargin,2*self.padding + self.dataSelectionHeight + self.imageXlabelMargin)
        self.dataSelectionBL = (self.imageAxesBL[0] - self.imageYlabelMargin,self.padding)
        
        self.layerSliderWidth = self.infoDisplayWidth - 2*self.padding*1.2
        self.layerSliderHeight = self.font*1.2/(self.figH*self.dpi)#rel
        self.layerSliderBL = (self.infoDisplayBL[0]+self.padding,self.infoDisplayBL[1]+self.padding)
        self.layerInfoWidth = self.layerSliderWidth
        self.layerInfoHeight = self.layerSliderHeight
        self.layerInfoBL = (self.layerSliderBL[0],self.layerSliderBL[1]+self.layerSliderHeight+self.padding)
        
        self.layerBottomHeight = self.layerInfoHeight*1.2
        self.layerBottomWidth = (self.layerInfoWidth-self.padding)/2.
        self.layerBottomBL = (self.layerSliderBL[0],self.layerInfoBL[1] + self.layerInfoHeight + self.padding)
        self.layerDownHeight = self.layerBottomHeight
        self.layerDownWidth = self.layerBottomWidth
        self.layerDownBL = (self.layerBottomBL[0]+self.layerBottomWidth+self.padding,self.layerBottomBL[1])
        
        self.layerTopHeight = self.layerBottomHeight
        self.layerTopWidth = self.layerBottomWidth
        self.layerTopBL = (self.layerBottomBL[0],self.layerBottomBL[1] + self.layerBottomHeight + self.padding)
        self.layerUpHeight = self.layerBottomHeight
        self.layerUpWidth = self.layerBottomWidth
        self.layerUpBL = (self.layerDownBL[0],self.layerTopBL[1])
        
        self.wavelengthHeight = self.layerBottomHeight
        self.wavelengthWidth = self.layerBottomWidth
        self.wavelengthBL = (self.layerDownBL[0],self.layerTopBL[1] + self.layerTopHeight + self.padding)
        self.frequencyHeight = self.layerBottomHeight
        self.frequencyWidth = self.layerBottomWidth
        self.frequencyBL = (self.wavelengthBL[0],self.wavelengthBL[1] + self.wavelengthHeight + self.padding)
        
        #print self.layerSliderBL,self.layerInfoBL,self.layerBottomBL,self.layerDownBL,self.layerTopBL,self.layerUpBL
        if self.infoDisplayAxes is None:
            self.infoDisplayAxes = plt.axes([self.infoDisplayBL[0],self.infoDisplayBL[1],self.infoDisplayWidth,self.infoDisplayHeight])
            self.infoDisplayAxes.patch.set_facecolor('silver')
            self.infoDisplayAxes.set_xticks([])
            self.infoDisplayAxes.set_yticks([])
            #self.infoDisplayAxes
            #set things in there
        else:
            self.infoDisplayAxes.set_position([self.infoDisplayBL[0],self.infoDisplayBL[1],self.infoDisplayWidth,self.infoDisplayHeight],which='both')
        
        if self.layerSliderAxes is None:
            self.layerSliderAxes = plt.axes([self.layerSliderBL[0],self.layerSliderBL[1],self.layerSliderWidth,self.layerSliderHeight])
            self.layerSliderAxes.patch.set_facecolor('silver')
            self.layerSliderAxes.set_frame_on(False)
            self.layerSliderAxes.set_xticks([])
            self.layerSliderAxes.set_yticks([])
            self.layerSlider = Slider(self.layerSliderAxes,"", self.minLayer, self.maxLayer, valinit=self.curLayer, valfmt='%d', closedmin=True, closedmax=True, slidermin=None, slidermax=None, dragging=True)
            self.layerSlider.on_changed(self.layerSliderCb)
        else:
            self.layerSliderAxes.set_position([self.layerSliderBL[0],self.layerSliderBL[1],self.layerSliderWidth,self.layerSliderHeight],which='both')
        
        if self.layerInfoAxes is None:
            self.layerInfoAxes = plt.axes([self.layerInfoBL[0],self.layerInfoBL[1],self.layerInfoWidth,self.layerInfoHeight])
            self.layerInfoAxes.patch.set_facecolor('silver')
            self.layerInfoAxes.set_frame_on(False)
            self.layerInfoAxes.set_xticks([])
            self.layerInfoAxes.set_yticks([])
            self.layerInfoBn = Button(self.layerInfoAxes,"Layer: {0} [{1}km]".format(self.curLayer,self.curLayerHeight), color='none',hovercolor='none')
        else:
            self.layerInfoAxes.set_position([self.layerInfoBL[0],self.layerInfoBL[1],self.layerInfoWidth,self.layerInfoHeight],which='both')
        
        if self.layerBottomAxes is None:
            self.layerBottomAxes = plt.axes([self.layerBottomBL[0],self.layerBottomBL[1],self.layerBottomWidth,self.layerBottomHeight])
            self.layerBottomAxes.patch.set_facecolor('silver')
            self.layerBottomAxes.set_frame_on(True)
            self.layerBottomAxes.set_xticks([])
            self.layerBottomAxes.set_yticks([])
            self.layerBottomBn = Button(self.layerBottomAxes,"Bottom", color='grey',hovercolor='white')
            self.layerBottomBn.on_clicked(self.layerBottomBnCb)
        else:
            self.layerBottomAxes.set_position([self.layerBottomBL[0],self.layerBottomBL[1],self.layerBottomWidth,self.layerBottomHeight],which='both')
        
        if self.layerDownAxes is None:
            self.layerDownAxes = plt.axes([self.layerDownBL[0],self.layerDownBL[1],self.layerDownWidth,self.layerDownHeight])
            self.layerDownAxes.patch.set_facecolor('silver')
            self.layerDownAxes.set_frame_on(True)
            self.layerDownAxes.set_xticks([])
            self.layerDownAxes.set_yticks([])
            self.layerDownBn = Button(self.layerDownAxes,"Down", color='grey',hovercolor='white')
            self.layerDownBn.on_clicked(self.layerDownBnCb)
        else:
            self.layerDownAxes.set_position([self.layerDownBL[0],self.layerDownBL[1],self.layerDownWidth,self.layerDownHeight],which='both')
        
        if self.layerTopAxes is None:
            self.layerTopAxes = plt.axes([self.layerTopBL[0],self.layerTopBL[1],self.layerTopWidth,self.layerTopHeight])
            self.layerTopAxes.patch.set_facecolor('silver')
            self.layerTopAxes.set_frame_on(True)
            self.layerTopAxes.set_xticks([])
            self.layerTopAxes.set_yticks([])
            self.layerTopBn = Button(self.layerTopAxes,"Top", color='grey',hovercolor='white')
            self.layerTopBn.on_clicked(self.layerTopBnCb)
        else:
            self.layerTopAxes.set_position([self.layerTopBL[0],self.layerTopBL[1],self.layerTopWidth,self.layerTopHeight],which='both')
        
        if self.layerUpAxes is None:
            self.layerUpAxes = plt.axes([self.layerUpBL[0],self.layerUpBL[1],self.layerUpWidth,self.layerUpHeight])
            self.layerUpAxes.patch.set_facecolor('silver')
            self.layerUpAxes.set_frame_on(True)
            self.layerUpAxes.set_xticks([])
            self.layerUpAxes.set_yticks([])
            self.layerUpBn = Button(self.layerUpAxes,"Up", color='grey',hovercolor='white')
            self.layerUpBn.on_clicked(self.layerUpBnCb)
        else:
            self.layerUpAxes.set_position([self.layerUpBL[0],self.layerUpBL[1],self.layerUpWidth,self.layerUpHeight],which='both')
        
        if self.wavelengthAxes is None:
            self.wavelengthAxes = plt.axes([self.wavelengthBL[0],self.wavelengthBL[1],self.wavelengthWidth,self.wavelengthHeight])
            self.wavelengthAxes.patch.set_facecolor('White')
            self.wavelengthAxes.set_frame_on(True)
            self.wavelengthAxes.set_xticks([])
            self.wavelengthAxes.set_yticks([])
            self.wavelengthTb = TextBox(self.wavelengthAxes,r"$\lambda$ (m)", initial = '%.3f'%(self.curWavelength),color='white',hovercolor='white', label_pad = self.padding)
            self.wavelengthTb.on_submit(self.wavelengthTbCb)
        else:
            self.wavelengthAxes.set_position([self.wavelengthBL[0],self.wavelengthBL[1],self.wavelengthWidth,self.wavelengthHeight],which='both')
        
        if self.frequencyAxes is None:
            self.frequencyAxes = plt.axes([self.frequencyBL[0],self.frequencyBL[1],self.frequencyWidth,self.frequencyHeight])
            self.frequencyAxes.patch.set_facecolor('White')
            self.frequencyAxes.set_frame_on(True)
            self.frequencyAxes.set_xticks([])
            self.frequencyAxes.set_yticks([])
            self.frequencyTb = TextBox(self.frequencyAxes,r"$\nu$ (MHz)", initial = "%d"%(1e-6*3e8/self.curWavelength),color='white',hovercolor='white', label_pad = self.padding)
            self.frequencyTb.on_submit(self.frequencyTbCb)
        else:
            self.frequencyAxes.set_position([self.frequencyBL[0],self.frequencyBL[1],self.frequencyWidth,self.frequencyHeight],which='both')
        
        
        if self.dataSelectionAxes is None:
            self.dataSelectionAxes = plt.axes([self.dataSelectionBL[0],self.dataSelectionBL[1],self.dataSelectionWidth,self.dataSelectionHeight])
            self.dataSelectionAxes.patch.set_facecolor('silver')
            self.dataSelectionRB = RadioButtons(self.dataSelectionAxes,self.dataSelection)
            for lab in self.dataSelectionRB.labels:
                lab.set_fontsize=self.font
            self.dataSelectionRB.on_clicked(self.updateDataSelector)
        else:
            self.dataSelectionAxes.set_position([self.dataSelectionBL[0],self.dataSelectionBL[1],self.dataSelectionWidth,self.dataSelectionHeight],which='both')
        
        if self.imageAxes is None:
            self.imageAxes = plt.axes([self.imageAxesBL[0],self.imageAxesBL[1],self.imageAxesWidth,self.imageAxesHeight])
            self.imageAxes.patch.set_facecolor('black')
            self.imageAxes.set_xticks([])
            self.imageAxes.set_yticks([])
            self.imageAxesCur = Cursor(self.imageAxes,horizOn=True, vertOn=True, color='red',lw=1)
        else:
            self.imageAxes.set_position([self.imageAxesBL[0],self.imageAxesBL[1],self.imageAxesWidth,self.imageAxesHeight],which='both')
        plt.draw()
        
    def updateDataSelector(self,label):
        '''Sets self.dataCaller which is a function which takes time,layer and returns (2D array, (xaxis,'units'), (yaxis,'units')'''
        dataCaller = self.dataSelectionCallers[label]
        if dataCaller == None:
            self.log("No attached data caller for: {0}".format(label))
            self.dataCaller = lambda time,layer: None,None,None#2D map
        else:
            self.dataCaller = dataCaller
    def layerSliderCb(self,sliderVal):
        self.curLayer = int(sliderVal)
        self.layerInfoBn.label.set_text("Layer: {0} [{1}km]".format(self.curLayer,self.getCurLayerHeight()))
        plt.draw()
    def layerBottomBnCb(self,event):
        #event.button is button used
        self.curLayer = 0
        self.layerSlider.set_val(self.curLayer)
    def layerDownBnCb(self,event):
        #event.button is button used
        self.curLayer = max(self.curLayer-1,0)
        self.layerSlider.set_val(self.curLayer)
    def layerTopBnCb(self,event):
        #event.button is button used
        self.curLayer = self.maxLayer
        self.layerSlider.set_val(self.curLayer)
    def layerUpBnCb(self,event):
        #event.button is button used
        self.curLayer = min(self.curLayer+1,self.maxLayer)
        self.layerSlider.set_val(self.curLayer)
    def wavelengthTbCb(self,text):
        self.log(text)
    def frequencyTbCb(self,text):
        self.log(text)
    def getCurLayerHeight(self):
        '''returns height of current layer in km.'''
        #use simTK eventually to get the height
        return self.curLayer*100.
gui = visual('Test Array')
gui.createGui()
plt.show()

Visualizing Array: Test Array
No attached data caller for: Intensity <E.E>
No attached data caller for: Optical Thickness
No attached data caller for: Electron Density


In [2]:
from matplotlib.widgets import MultiCursor
from pylab import figure, show, np

t = np.arange(0.0, 2.0, 0.01)
s1 = np.sin(2*np.pi*t)
s2 = np.sin(4*np.pi*t)
fig = figure()
ax1 = fig.add_subplot(211)
ax1.plot(t, s1)

ax2 = fig.add_subplot(212, sharex=ax1)
ax2.plot(t, s2)

multi = MultiCursor(fig.canvas, (ax1, ax2), color='r', lw=1,
                    horizOn=False, vertOn=True)
show()